In [1]:
import sys
sys.path.append(r'C:\Users\kuina\OneDrive\TFG\Codigo')

In [2]:
from datasets import load_dataset
import json

from utils_creacion_datasets import create_dependency_tags

from my_utils import text_to_num,num_to_text

## Datos Huggingface

In [3]:
convai2_about = load_dataset('md_gender_bias','convai2_inferred')
train_about = convai2_about['train']
print(train_about[0])

Reusing dataset md_gender_bias (C:\Users\kuina\.cache\huggingface\datasets\md_gender_bias\convai2_inferred\1.0.0\8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': "hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape .", 'binary_label': 1, 'binary_score': 0.6521999835968018, 'ternary_label': 2, 'ternary_score': 0.4496000111103058}


In [4]:
train_about = train_about.remove_columns(['binary_score', 'ternary_label', 'ternary_score'])

In [5]:
train_about

Dataset({
    features: ['text', 'binary_label'],
    num_rows: 131438
})

In [9]:
#Filtro repetidos para que sean una única entrada (Aqui aun no estoy filtrando las etiquetas)
data = {}
print(f'Elementos antes de filtrar repetidos = {len(train_about)}')
for item in train_about:
    text = item['text']
    label = item['binary_label']
    if text not in data:
        # data[text] =[label]
        data[text] =[num_to_text['about'][label]]
    else:
        data[text].append(num_to_text['about'][label])

print(f'Elementos después de filtrar repetidos = {len(data)}')

# Filtro las etiquetas para que no haya items repetidos

for item in data:
    data[item] = list(set(data[item]))


Elementos antes de filtrar repetidos = 131438
Elementos después de filtrar repetidos = 115877


## Carga datos ParlAI

In [10]:
with open(r'./convai_two_dataset.json') as f: #Este tiene unks
    convai2_parlai = json.load(f)

# Unión de los datasets

Me interesa tener 3 etiquetas, por lo que quiero las que oraciones que salgan en convai2_parlai y en data (etiquetas about the convai2). Itero sobre convai2_parlai porque es el conjunto más pequeño de los 2

In [22]:
join_data = {}
for item in convai2_parlai:
    if item in data:
        join_data[item] = {'label_about':data[item],
                            'label_to':convai2_parlai[item]['partner'],
                            'label_as':convai2_parlai[item]['self']}


In [35]:
join_data["good , how are you ?"]

{'label_about': ['ABOUT:male'],
 'label_to': ['PARTNER:unknown'],
 'label_as': ['SELF:female', 'SELF:unknown']}

In [36]:
with open('convai_all_labels.json','w') as f:
    json.dump(join_data,f)

# Alternativa: parsing solo con Stanza

In [9]:
with open('convai_all_labels.json','r') as f:
    data = json.load(f)

In [10]:
index_errors = create_dependency_tags(data,['absolute','relative','pos'],'_')

Comenzando parsing de dependencias....

Parsing de dependencias terminado. Duración del proceso: 3.1797553579012554 minutos
Comenzando a generar el encoding para las dependencias


100%|██████████| 73398/73398 [00:06<00:00, 12097.91it/s]

Proceso terminado con éxito


In [11]:
len(index_errors)

20127

In [12]:
with open('convai_complete.json','w') as f:
    json.dump(data,f)

In [14]:
index_errors

[([1	sometimes	sometimes	ADV	RB	_	3	advmod	_	_,
   2	i	I	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	3	nsubj	_	_,
   3	do	do	VERB	VBP	Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin	0	root	_	_,
   4	.	.	PUNCT	.	_	3	punct	_	_,
   5	while	while	SCONJ	IN	_	7	mark	_	_,
   6	i'm	i'm	ADV	RB	_	7	advmod	_	_,
   7	asleep	asleep	ADJ	JJ	Degree=Pos	3	advcl	_	_],
  'pos'),
 ([1	is	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	2	cop	_	_,
   2	there	there	ADV	RB	PronType=Dem	6	advmod	_	_,
   3	?	?	PUNCT	.	_	2	punct	_	_,
   4	i'm	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	6	cop	_	_,
   5	not	not	PART	RB	_	6	advmod	_	_,
   6	sure	sure	ADJ	JJ	Degree=Pos	0	root	_	_,
   7	of	of	ADP	IN	_	8	case	_	_,
   8	that	that	PRON	DT	Number=Sing|PronType=Dem	6	obl	_	_],
  'pos'),
 ([1	wow	wow	INTJ	UH	_	4	discourse	_	_,
   2	that	that	PRON	DT	Number=Sing|PronType=Dem	4	nsubj	_	_,
   3	's	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	4	cop	_	_,
  